# STATS 509 HW2
### unique name: tangsw, umid:31975136

# Q1
## a 


$\mathbb{P}(R < -V \tilde{a}R) = q $

$F(-V \tilde{a}R) = q < 0.5$

Therefore, $-V \tilde{a}R<\mu$ since double exponential distribution is symmetric on $\mu$

$\frac{1}{2}exp(\lambda(-V \tilde{a}R-\mu))=q$

$V \tilde{a}R = -\mu - \frac{ln(2q)}{\lambda}$

$VaR = V \tilde{a}R \times Price  = -\mu - \frac{ln(2q)}{\lambda} \times Price$

## b

Since $V\tilde{a}R = -\mu - \frac{ln(2q)}{\lambda}$,and q < 0; $\lambda > 0 $ and $V\tilde{a}R$ is strictly bigger than -$\mu$. 

Loss X ~ DExp($-\mu, \lambda$), Relative ES = $\frac{1}{q}\int_{V\tilde{a}R_q}^{\infty}x\frac{1}{2}\lambda exp(-\lambda (x+\mu))dx$

$ES = \frac{1}{2q}exp(-\lambda(V\tilde{a}R+\mu))(V\tilde{a}R + \frac{1}{\lambda})$

From a) we know $V \tilde{a}R = -\mu - \frac{ln(2q)}{\lambda}$, plug this is 

Gives us $ES = V \tilde{a}R + \frac{1}{\lambda}= -\mu - \frac{ln(2q)}{\lambda}+ \frac{1}{\lambda}$




## c

In [1]:
source("startup.R")
df = read.csv("Nasdaq_daily_Jan1_2019-Dec31_2021.csv")
head(df)

,Date,Open,High,Low,Close,Adj.Close,Volume
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2019-01-02,6506.91,6693.71,6506.88,6665.94,6665.94,2261800000
2,2019-01-03,6584.77,6600.21,6457.13,6463.50,6463.50,2607290000
3,2019-01-04,6567.14,6760.69,6554.24,6738.86,6738.86,2579550000
4,2019-01-07,6757.53,6855.60,6741.40,6823.47,6823.47,2507550000
5,2019-01-08,6893.44,6909.58,6795.86,6897.00,6897.00,2380290000
6,2019-01-09,6923.06,6985.22,6899.56,6957.08,6957.08,2422590000


**Note the the MLE of location parameter $\mu$ is the sample median and scale parameter $1/\lambda$ is the sample mean deviation from the median**

Source: [https://stats.stackexchange.com/questions/281682/how-to-fit-a-data-against-a-laplace-double-exponential-distribution-and-check](https://stats.stackexchange.com/questions/281682/how-to-fit-a-data-against-a-laplace-double-exponential-distribution-and-check)


In [2]:
n = length(df$Adj.Close)
R = df$Adj.Close[-1]/df$Adj.Close[-n]-1
m = median(R)
data.frame(m = m, lambda = 1/mean(abs(R - m)))
l = 1/mean(abs(R- m))

m,lambda
<dbl>,<dbl>
0.001913407,99.42223


$\hat{\mu} = 0.001913407$, and $\hat{\lambda} = 99.4222$

In [3]:
var_rl = -1 * qdexp(p = 0.01, mu = m, lambda = l)
print(var_rl)
var = 10^7 * var_rl
print(c('Relative Var is:',var_rl))
print(c('Var is:',round(var,2)))
p = 0.01
ES = 1e7*(var_rl-1/l)
print(c("Estimated Shortfall is:",round(ES,2)))

[1] 0.03743416
[1] "Relative Var is:"   "0.0374341607656987"
[1] "Var is:"   "374341.61"
[1] "Estimated Shortfall is:" "273760.48"              


The probability of loss less than 374341.61 usd is 0.99. 

Given that a loss is occuring at or greater than \\$ 374341.61, the mean loss of this porfolio is \$ 273760.48.
## d

#### using MLE estimator of $\mu, \sigma$

$\mu = sample mean, \hat{\sigma}^2 = \frac{n}{n-1}sample variance $

In [5]:
m = mean(R)
variance = var(R) * n / (n-1)
sd = sqrt(variance)
print(c('mu and sigma:',m,sd))
var_rl = qnorm(p, m, sd) * -1
print(c("Relative VaR is :",var_rl))
var = 10^7 * var_rl
print(c("VaR is :",round(var,2)))
ES = -m + sd / p *(dnorm((qnorm(p = p, mean = 0, sd = 1)),0,1))
print(ES*1e7)

[1] "mu and sigma:"       "0.00126080759968301" "0.0155792444898162" 
[1] "Relative VaR is :"  "0.0349819346983633"
[1] "VaR is :"  "349819.35"
[1] 402612.2


$\hat{\mu} = 0.0012608$, and $\hat{\sigma} = 0.015579$

The value at risk is 349819.35 usd. It means the probability of loss less than $ 349819.35 is 0.99.

The Expected shortfall is 402612.2 usd , which means given a loss at or below 349819.35, the mean loss of this portfolio is  402612.2 usd. 

## Try to compare the kurtosis and skewness

In [6]:
library("moments")
sk = skewness(R)
kt = kurtosis(R)-3 # to excessive kurtosis
print(paste("True Return Skewness is ",sk))
print(paste("True Return Kurtosis is ",kt))
sk1 = 0
kt1 = 3
sk2 = 0
kt2 = 6
data.frame(skewness = c(sk, sk1, sk2),kurtosis = c(kt,kt1,kt2),row.names = c("True","Normal","Double Exponential"))


[1] "True Return Skewness is  -0.743521849028927"
[1] "True Return Kurtosis is  11.1955040430353"


,skewness,kurtosis
,<dbl>,<dbl>
True,-0.7435218,11.1955
Normal,0.0000000,3.0000
Double Exponential,0.0000000,6.0000


Based on the Skewness and kurtosis, Double exponential distribution is closer to the True distribution, which should provide more accurate result upon prediction.

# Q2
## a

From the prompt, we know that $F(\mu) = 0.9$

Expand the conditional probability by joint dividing marginal probability.

$\mathbb{P}(X \leq x|X \geq \mu) = \frac{\mathbb{P}(\mu \leq X \leq x)}{\mathbb{P}(X \geq \mu)} = \frac{F(x)-F(\mu)}{1- F(\mu)}$, where F is the c.d.f of r.v. X.

This gives us 

$\frac{F(x) - 0.9}{0.1} = 1 - (1+\frac{\xi(x-\mu)}{\sigma})^{-1/\xi}, x \geq \mu$

Solve for F(x) will give us the c.d.f of X


F(x) = \begin{cases}
1 - 0.1(1+\frac{\xi(x-\mu)}{\sigma})^{-1/\xi} & \text{ if } x \geq \mu \\
0.9 & \text{ if } x < \mu
\end{cases}

By definition X is loss

$\mathbb{P}(X>V\tilde{a}R) = q = 0.01$

$V\tilde{a}R =  F^{-1}(0.01) = Q(0.01)$ , where Q denotes the quantile function of F(x)

$V\tilde{a}R = \mu + \frac{\sigma (10^\xi - 1)}{\xi}$

## b 
$ES = \mathbb{P}(X<x|X>V\tilde{a}R) = \frac{F(x) - (1-q)}{q} = 1 - 10(1+\frac{\xi(x-\mu)}{\sigma})^{-1/\xi}, x \geq V\tilde{a}R $

This is equivalent to 

= $1 - (1+ \frac{\xi(x-\mu+\frac{\sigma}{\xi}-\frac{10^\xi}{\xi})}{10^\xi\sigma})^{-1/\xi}$

If we let $\sigma' = 10^{\xi}\sigma, \mu' = \mu + \frac{10^\xi \sigma- \sigma}{\xi}$, then we can rewrite the shortfall distribution into a Generalized Pareto distribution.

$F_{\mathbf{ES}}(x) =1 - (1+\frac{\xi(x-\mu')}{\sigma'})^{-1/\xi}, if x>\mu'$

## c
Because ES ~ GPD($\mu',\sigma',\xi$), 

$\mathbb{E}(ES) = \mu' + \frac{\sigma'}{1- \xi} = \mu + \frac{10^\xi \sigma- \sigma}{\xi} + \frac{10^{\xi} \sigma}{1- \xi}$